__ResNet18__ \
하나의 메뉴만 있는 사진으로 모델 학습 진행 \
런타임 종료로 인해 epoch6에서 중단 \
학습결과 Loss: 0.3156, Validation Accuracy: 74.52%

### 1. 데이터 불러오기

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# 경로 설정
folder_path = '/content/drive/MyDrive/dataset/empty_labeling'

# 파일 개수 세기
if os.path.exists(folder_path):
    num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    print(f"Total number of files: {num_files}")
else:
    print(f"The folder path {folder_path} does not exist.")

### 2. 이미지-라벨링 매핑

In [ ]:
import shutil
import os

output_dir = '/content/drive/MyDrive'
resnet_dataset_path = '/content/drive/MyDrive/resnet_dataset'


# 압축 풀린 데이터셋 경로 설정
dataset_path = os.path.join(output_dir, 'dataset/kfood')  # kfood 폴더 경로
output_txts_path = os.path.join(output_dir, 'dataset/empty_labeling')  # empty_labeling 폴더 경로

# 대분류 폴더 순회
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)

    if os.path.isdir(category_path):
        # 중분류 폴더 순회
        for subcategory in os.listdir(category_path):
            subcategory_path = os.path.join(category_path, subcategory)

            if os.path.isdir(subcategory_path):
                # 이미지와 properties 파일을 포함한 파일 탐색
                for img_file in os.listdir(subcategory_path):
                    if img_file.endswith(('.jpg', '.png', '.jpeg')):
                        img_path = os.path.join(subcategory_path, img_file)

                        # 좌표 파일을 empty_labeling에서 찾기
                        base_name = os.path.splitext(img_file)[0]
                        txt_file = f'{base_name}.txt'
                        txt_file_path = os.path.join(output_txts_path, txt_file)

                        # 텍스트 파일이 있는 경우에만 복사
                        if os.path.exists(txt_file_path):
                            print(f"Copying image: {img_path}")
                            shutil.copy(img_path, resnet_dataset_path)
                            print(f"Copying label: {txt_file_path}")
                            shutil.copy(txt_file_path, resnet_dataset_path)
                        else:
                            print(f"Label not found for: {img_file}, skipping image.")

### 3. 모델링

In [ ]:
# ResNet18 모델링 코드

from PIL import Image
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet18

# empty_labeling 폴더에 있는 txt 파일 이름을 불러오는 함수
def get_valid_image_names(empty_labeling_dir):
    valid_image_names = set()
    for file in os.listdir(empty_labeling_dir):
        if file.endswith('.txt'):
            # .txt를 제거하고 이미지 파일 이름으로 사용
            image_name = file.replace('.txt', '')
            valid_image_names.add(image_name)
    return valid_image_names

# 이미지 경로와 라벨을 자동으로 생성하는 함수
def create_image_label_list(root_dir, valid_image_names):
    image_paths = []
    labels = []
    label_dict = {}
    label_id = 0

    for root, dirs, files in os.walk(root_dir):
        # 각 폴더를 카테고리로 처리
        if files:
            category = os.path.basename(root)
            if category not in label_dict:
                label_dict[category] = label_id
                label_id += 1
            # 파일이 있는 경우에만 이미지 경로와 라벨 추가
            for file in files:
                # 이미지가 valid_image_names에 있을 때만 처리
                if file.lower().endswith(('.jpg', '.jpeg', '.png')) and os.path.splitext(file)[0] in valid_image_names:
                    image_paths.append(os.path.join(root, file))
                    labels.append(label_dict[category])

    return image_paths, labels, label_dict

# 데이터셋 클래스 정의
class KoreanFoodDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# ResNet 모델 설정
def get_resnet_model(num_classes):
    model = resnet18(pretrained=True)
    in_features = model.fc.in_features
    model.fc = torch.nn.Linear(in_features, num_classes)
    return model

# 학습 및 평가 함수
def train_model(model, train_loader, valid_loader, num_epochs, device):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    best_accuracy = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        validation_acc = 100 * correct / total if total > 0 else 0
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Validation Accuracy: {validation_acc:.2f}%")

        # 성능이 더 좋으면 저장
        if validation_acc > best_accuracy:
            best_accuracy = validation_acc
            best_model_wts = model.state_dict()

    # 성능이 가장 좋았던 모델 저장
    torch.save(best_model_wts, '/content/drive/MyDrive/korean_food_resnet_best.pth')

# 경로 설정 및 데이터 준비
empty_labeling_dir = '/content/drive/MyDrive/dataset/empty_labeling'
root_dir = '/content/drive/MyDrive/dataset/kfood'

# valid_image_names 생성
valid_image_names = get_valid_image_names(empty_labeling_dir)

# 이미지 경로와 라벨 생성
image_paths, labels, label_dict = create_image_label_list(root_dir, valid_image_names)
print(f"총 {len(image_paths)}개의 유효한 이미지가 발견되었습니다.")

# 이미지 크기 조정 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet의 기본 입력 크기는 224x224
    transforms.ToTensor()
])

# 데이터셋 및 DataLoader 준비
dataset = KoreanFoodDataset(image_paths, labels, transform=transform)

# 데이터셋 나누기 (데이터가 충분하지 않은 경우 나누기 어려움)
train_size = int(0.7 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# 모델 설정 및 학습
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = len(label_dict)  # 카테고리 수
model = get_resnet_model(num_classes)
model.to(device)

# 학습 시작
train_model(model, train_loader, valid_loader, num_epochs=10, device=device)

# 학습된 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/korean_food_resnet_full.pth')

총 148244개의 유효한 이미지가 발견되었습니다.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/p

Epoch [1/10], Loss: 1.9127, Validation Accuracy: 67.23%
Epoch [2/10], Loss: 1.1009, Validation Accuracy: 70.88%
Epoch [3/10], Loss: 0.8211, Validation Accuracy: 70.32%
Epoch [4/10], Loss: 0.6183, Validation Accuracy: 73.64%
Epoch [5/10], Loss: 0.4499, Validation Accuracy: 74.00%
Epoch [6/10], Loss: 0.3156, Validation Accuracy: 74.52%


KeyboardInterrupt: 

__런타임이 끊겨 epoch 6에서 학습이 중단__

In [ ]:
def load_model(model_path, num_classes, device):
    model = get_resnet_model(num_classes)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()  # 평가 모드로 설정 (학습 중 적용되었던 dropout 등을 비활성화)
    return model

def evaluate_model(model, test_loader, device):
    model.eval()  # 평가 모드로 설정
    total = 0
    correct = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():  # 평가 시에는 gradient 계산을 하지 않음
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # 실제 라벨과 예측 라벨 저장 (추후 성능 지표 계산을 위해)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total if total > 0 else 0
    print(f"테스트 데이터에서 정확도: {accuracy:.2f}%")
    return all_labels, all_preds

In [ ]:
# 예시: 검증 데이터셋을 테스트 데이터셋으로 재사용
test_loader = valid_loader  # 검증 데이터셋을 테스트 데이터셋으로 사용 가능

# 별도로 테스트 데이터가 있다면, 해당 데이터셋을 DataLoader로 로드
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 모델 경로와 평가할 데이터 준비
model_path = '/content/drive/MyDrive/korean_food_resnet_best.pth'
num_classes = len(label_dict)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 학습된 모델 불러오기
model = load_model(model_path, num_classes, device)

# 평가 수행
all_labels, all_preds = evaluate_model(model, test_loader, device)
